In [ ]:
albums = ['David Bowie (1967)', 
          'David Bowie (a.k.a. Space Oddity) (1969)', 
          'The Man Who Sold the World (1970)',
          'Hunky Dory (1971)',
          'The Rise and Fall of Ziggy Stardust and the Spiders from Mars (1972)',
          'Aladdin Sane (1973)',
          'Pin Ups (1973)',
          'Diamond Dogs (1974)',
          'Young Americans (1975)',
          'Station to Station (1976)',
          'Low (1977)',
          '“Heroes” (1977)',
          'Lodger (1979)',
          'Scary Monsters (and Super Creeps) (1980)',
          "Let’s Dance (1983)",
          'Tonight (1984)',
          'Never Let Me Down (1987)',
          'Black Tie White Noise (1993)',
          'The Buddha of Suburbia (1993)',
          '1. Outside (1995)',
          'Earthling (1997)',
          '‘hours...’ (1999)',
          'Heathen (2002)',
          'Reality (2003)',
          'The Next Day (2013)',
          '★ (Blackstar) (2016)']

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import re
import lyricsgenius
import json

In [ ]:
key = 'AhSIl1-8gPr2DZk9epSuJT03xn-ti9Sbv84EeThoHsQPELBOkqlKRmOca2BWs0TF'

In [ ]:
driver = webdriver.Chrome()
url = 'https://genius.com/artists/David-bowie/albums'
driver.get(url)

scroll_amount = 1000
target_article_xpath = '//*[@id="application"]/main/div[1]/div/div[3]/div[3]/div/div[2]/ul/li[127]/a/div[2]/h3'
while True:
    try:
        target_article = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, target_article_xpath))
        )
        print("Target article found!")
        break  # Break the loop if the target article is found
    except:
        # If the target article is not found within the timeout, scroll down and retry
        print("Target article not found. Scrolling down and retrying...")
        driver.execute_script(f"window.scrollTo(0, {scroll_amount});")
        scroll_amount+=1300
driver.execute_script(f"window.scrollTo(0, 0);")
search_album = driver.find_elements(By.TAG_NAME, "li")

In [ ]:
matched_albums = []
pattern = re.compile(r'(.+)\n.*(\d{4})')
for el in search_album: 
    matches = pattern.finditer(el.text)
    for match in matches:
        album_name = match.group(1)
        album_year = match.group(2)
        result = f'{album_name} ({album_year})'
        for album in albums:
            if(result==album):
                matched_albums.append(el)

In [ ]:
genius = lyricsgenius.Genius(key)

In [ ]:
bowie_data = []
for matched_album in matched_albums:
    title_element = matched_album.find_element(By.TAG_NAME, 'a')
    album_name = matched_album.text.split('\n',1)[0]
    album_date = matched_album.text.split('\n',1)[1].split(' ')[-1]
    
    
    title_element.send_keys(Keys.CONTROL + Keys.RETURN)
    driver.switch_to.window(driver.window_handles[-1])
    
    time.sleep(5)

    search_songs = driver.find_elements(By.TAG_NAME, 'album-tracklist-row')
    songs = []
    
    for song in search_songs:
        song_name = song.find_element(By.TAG_NAME, 'h3')
        song_name = list(song_name.text.split(' '))[:-1]
        song_name = ' '.join(song_name)
        
        song_lyrics = genius.search_song(song_name, 'David Bowie')
        if(song_lyrics!=None):
            song_lyrics=song_lyrics.lyrics
        else:
            song_lyrics=0
        time.sleep(5)
        
        songs.append({"Name": song_name, "Lyrics": song_lyrics}) 
        
    bowie_data.append({
        "Name": album_name,
        "Year": album_date,
        "Songs": songs
    })

    driver.close()
    driver.switch_to.window(driver.window_handles[0])
driver.close()

In [ ]:
filename = "bowie_data.json"

with open(filename, 'w') as file:
    json.dump(bowie_data, file, indent=4)

print(f"Data successfully saved to {filename}")